In [1]:
import pandas as pd 
import numpy as np

# Para garantir que o notebook encontre a pasta de dados corretamente 
import os 
# Imports para pré-processamento e modelagem com Scikit-learn 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV



# Imports para MLOps e salvamento de artefatos 
import mlflow 
import mlflow.sklearn 
import joblib 
import shap

# Ponto de atenção: O notebook está em 'notebooks/', então subimos um nível para encontrar 'data/' 
DATA_PATH = os.path.join("..", "data", "churn-bigml-80.csv") 
MODEL_DIR = os.path.join("..", "app")

# Garantir o salvamento na raiz 
from pathlib import Path

# Constrói o caminho para a pasta raiz do projeto (subindo um nível a partir de 'notebooks/') # No contexto de um notebook, __file__ não existe, então usamos uma alternativa 
try: 
    # Esta linha funciona quando executado como script 
    project_root = Path(__file__).resolve().parents[1] 
except NameError: 
    # Esta linha funciona dentro de um notebook interativo 
    project_root = Path(os.getcwd()).resolve().parents[0] 

# Define o local onde a pasta 'mlruns' deve ser criada. 
# O 'file:' é o prefixo que o MLflow usa para indicar um caminho local.
tracking_uri = "file:" + str(project_root/"mlruns")

# Define este caminho como o local padrão para o MLflow salvar tudo. 
mlflow.set_tracking_uri(tracking_uri) 
print(f"MLflow tracking URI configurado para: {tracking_uri}")

MLflow tracking URI configurado para: file:C:\git\Analise_Churn\mlruns


In [2]:
# Célula 2: Carga e Análise Exploratória dos Dados (EDA) # O primeiro passo de qualquer projeto de ciência de dados é entender o material com o qual estamos trabalhando. # %% 
# Carregar o dataset 
try: 
    df = pd.read_csv(DATA_PATH) 
    print("Dataset carregado com sucesso!") 
except FileNotFoundError: 
    print(f"Erro: Arquivo não encontrado em '{DATA_PATH}'. Verifique o caminho.") 

#Renomeando as colunas
rename_map = {
    'State': 'Estado',
    'Account length': 'TempoConta',
    'Area code': 'CodigoArea',
    'International plan': 'PlanoInternacional',
    'Voice mail plan': 'PlanoCorreioVoz',
    'Number vmail messages': 'NumerodeMensagensdeVoz',
    'Total day minutes': 'TotalMinutosDia',
    'Total day calls': 'TotalChamadasDia',
    'Total day charge': 'CustoTotalDia',
    'Total eve minutes': 'TotalMinutosTardeNoite',
    'Total eve calls': 'TotalChamadasTardeNoite',
    'Total eve charge': 'CustoTotalTardeNoite',
    'Total night minutes': 'TotalMinutosNoturno',
    'Total night calls': 'TotalChamadasNoturno',
    'Total night charge': 'CustoTotalNoturno',
    'Total intl minutes': 'TotalMinutosInternacionais',
    'Total intl calls': 'TotalChamadasInternacionais',
    'Total intl charge': 'CustoTotalInternacional',
    'Customer service calls': 'ChamadasSuporte',
    'Churn': 'Churn'
}
df.rename(columns=rename_map, inplace=True)

# Visualizar as primeiras linhas print("--- Primeiras 5 linhas do dataset ---") 
display(df.head()) 
# Obter informações sobre os tipos de dados e valores nulos 
print("\n--- Informações do Dataset (tipos e nulos) ---") 
df.info() 


# Removemos as poucas linhas que ficaram com valor nulo após a conversão 
df.dropna(inplace=True) 
# Converter a coluna alvo 'Churn' para formato numérico (0 ou 1) 
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == True else 0)

print("\n--- Dataset após limpeza e conversão da coluna 'Churn' ---") 
display(df.head()) 
df.info()

Dataset carregado com sucesso!


,Estado,TempoConta,CodigoArea,PlanoInternacional,PlanoCorreioVoz,NumerodeMensagensdeVoz,TotalMinutosDia,TotalChamadasDia,CustoTotalDia,TotalMinutosTardeNoite,TotalChamadasTardeNoite,CustoTotalTardeNoite,TotalMinutosNoturno,TotalChamadasNoturno,CustoTotalNoturno,TotalMinutosInternacionais,TotalChamadasInternacionais,CustoTotalInternacional,ChamadasSuporte,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False



--- Informações do Dataset (tipos e nulos) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Estado                       2666 non-null   object 
 1   TempoConta                   2666 non-null   int64  
 2   CodigoArea                   2666 non-null   int64  
 3   PlanoInternacional           2666 non-null   object 
 4   PlanoCorreioVoz              2666 non-null   object 
 5   NumerodeMensagensdeVoz       2666 non-null   int64  
 6   TotalMinutosDia              2666 non-null   float64
 7   TotalChamadasDia             2666 non-null   int64  
 8   CustoTotalDia                2666 non-null   float64
 9   TotalMinutosTardeNoite       2666 non-null   float64
 10  TotalChamadasTardeNoite      2666 non-null   int64  
 11  CustoTotalTardeNoite         2666 non-null   float64
 12  TotalMinutosNoturno         

,Estado,TempoConta,CodigoArea,PlanoInternacional,PlanoCorreioVoz,NumerodeMensagensdeVoz,TotalMinutosDia,TotalChamadasDia,CustoTotalDia,TotalMinutosTardeNoite,TotalChamadasTardeNoite,CustoTotalTardeNoite,TotalMinutosNoturno,TotalChamadasNoturno,CustoTotalNoturno,TotalMinutosInternacionais,TotalChamadasInternacionais,CustoTotalInternacional,ChamadasSuporte,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Estado                       2666 non-null   object 
 1   TempoConta                   2666 non-null   int64  
 2   CodigoArea                   2666 non-null   int64  
 3   PlanoInternacional           2666 non-null   object 
 4   PlanoCorreioVoz              2666 non-null   object 
 5   NumerodeMensagensdeVoz       2666 non-null   int64  
 6   TotalMinutosDia              2666 non-null   float64
 7   TotalChamadasDia             2666 non-null   int64  
 8   CustoTotalDia                2666 non-null   float64
 9   TotalMinutosTardeNoite       2666 non-null   float64
 10  TotalChamadasTardeNoite      2666 non-null   int64  
 11  CustoTotalTardeNoite         2666 non-null   float64
 12  TotalMinutosNoturno          2666 non-null   float64
 13  TotalChamadasNotur

In [3]:
# Célula 3: Definição das Features e do Alvo
# Separamos nossas variáveis entre as que usaremos para prever (features) e a que queremos prever (alvo). # %% 

# A coluna 'customerID' não é útil para a predição, então a removemos 
#df = df.drop('customerID', axis=1) 

# X contém as features (todas as colunas exceto 'Churn') 
X = df.drop('Churn', axis=1) 
# y contém a variável alvo ('Churn') 
y = df['Churn'] 

# Separar os dados em conjuntos de treino e teste para avaliar o modelo de forma justa 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 
print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras") 
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

Tamanho do conjunto de treino: 2132 amostras
Tamanho do conjunto de teste: 534 amostras


In [4]:
#Célula 4: Pré-processamento com Pipelines 
# Machine Learning só funciona com números. Aqui, criamos um "pipeline" 
# para transformar nossas colunas de texto e numéricas 
# em um formato que o modelo entenda. Usar um pipeline garante
#  que o mesmo processamento seja aplicado de forma consistente.

# Identificar colunas numéricas e categóricas 
numeric_features = X.select_dtypes(include=np.number).columns.tolist() 
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()


print("numeric_features:",numeric_features)

print("categorical_features:",categorical_features)

# Criar transformadores para cada tipo de coluna 
# # Para numéricas: Padronizamos a escala (importante para muitos modelos) 
numeric_transformer = StandardScaler() 

# Para categóricas: Convertemos em colunas binárias (0 ou 1)
# handle_unknown='ignore' evita erros se uma categoria nova aparecer na predição 
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Juntar os transformadores em um único pré-processador 
preprocessor = ColumnTransformer( transformers=[ ('num', numeric_transformer, numeric_features), ('cat', categorical_transformer, categorical_features) ]) 
print("Pipeline de pré-processamento criado com sucesso.")


numeric_features: ['TempoConta', 'CodigoArea', 'NumerodeMensagensdeVoz', 'TotalMinutosDia', 'TotalChamadasDia', 'CustoTotalDia', 'TotalMinutosTardeNoite', 'TotalChamadasTardeNoite', 'CustoTotalTardeNoite', 'TotalMinutosNoturno', 'TotalChamadasNoturno', 'CustoTotalNoturno', 'TotalMinutosInternacionais', 'TotalChamadasInternacionais', 'CustoTotalInternacional', 'ChamadasSuporte']
categorical_features: ['Estado', 'PlanoInternacional', 'PlanoCorreioVoz']
Pipeline de pré-processamento criado com sucesso.


In [5]:
# Célula 5: Treinamento, Avaliação e Rastreamento com MLflow
# Este é o coração da fase. Treinamos o modelo, avaliamos seu desempenho e registramos tudo no MLflow 
# para garantir a reprodutibilidade.

# Definir o nome do experimento no MLflow 
mlflow.set_experiment("Predicao_Churn") 
# Iniciar uma execução do MLflow
with mlflow.start_run() as run: 
    print(f"Iniciando run do MLflow: {run.info.run_name}")

    # 1. Definir a grade de hiperparâmetros
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [5, 8, 12, None],
        'classifier__min_samples_leaf': [1, 3, 5],
        'classifier__random_state': [42]
    }


    # 2. Criar o modelo 
    model = RandomForestClassifier()
    
     # 3. Criar o pipeline completo: pré-processamento seguido do modelo 
    full_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

    # 4. Configurar o GridSearchCV
    grid_search = GridSearchCV(
        estimator=full_pipeline,
        param_grid=param_grid,
        cv=5,                # validação cruzada com 5 folds
        scoring='accuracy',  # métrica principal
        n_jobs=-1            # usar todos os núcleos disponíveis
    )


    #5. Treinar o pipeline completo print("Treinando o modelo...") 
    print("Treinando o modelo com GridSearchCV...")
    grid_search.fit(X_train, y_train)
    print("Treinamento concluído.")

    # Melhor combinação encontrada
    best_params = grid_search.best_params_
    print(f"Melhores hiperparâmetros: {best_params}")

    # 5. Fazer previsões com o melhor modelo
    y_pred = grid_search.predict(X_test)

    # 6. Calcular métricas 
    accuracy = accuracy_score(y_test, y_pred) 
    precision = precision_score(y_test, y_pred) 
    recall = recall_score(y_test, y_pred) 
    f1 = f1_score(y_test, y_pred) 
    print(f"\nAcurácia: {accuracy:.4f}") 
    print(f"Precisão: {precision:.4f}") 
    print(f"Recall: {recall:.4f}") 
    print(f"F1-Score: {f1:.4f}")

    # 7. Registrar tudo no MLflow 
    print("\nRegistrando parâmetros e métricas no MLflow...") 
    mlflow.log_params(best_params) 
    mlflow.log_metric("accuracy", accuracy) 
    mlflow.log_metric("precision", precision) 
    mlflow.log_metric("recall", recall) 
    mlflow.log_metric("f1_score", f1)

    # Salvar o pipeline completo como um "artefato" do MLflow 
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")
    print("Modelo registrado no MLflow com sucesso.")


    # 8. Salvar o pipeline para a nossa API 
    #Garantir que o diretório de destino exista 
    os.makedirs(MODEL_DIR, exist_ok=True) 
    pipeline_path = os.path.join(MODEL_DIR, "pipeline_churn.joblib") 
    joblib.dump(full_pipeline, pipeline_path) 
    print(f"Pipeline salvo para a API em: '{pipeline_path}'") 
    
print("\nExecução do MLflow concluída.")

c:\git\Analise_Churn\venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


Iniciando run do MLflow: bustling-bee-13
Treinando o modelo com GridSearchCV...
Treinamento concluído.
Melhores hiperparâmetros: {'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__n_estimators': 200, 'classifier__random_state': 42}

Acurácia: 0.9345
Precisão: 0.9778
Recall: 0.5641
F1-Score: 0.7154

Registrando parâmetros e métricas no MLflow...


2025/12/03 19:40:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/03 19:40:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo registrado no MLflow com sucesso.
Pipeline salvo para a API em: '..\app\pipeline_churn.joblib'

Execução do MLflow concluída.


In [6]:
# Célula 6: Seleção e Salvamento do Melhor Modelo para a API. 
# --- PASSO 1: COLE O 'RUN ID' DO SEU MELHOR MODELO AQUI --- 
# Após rodar a Célula 5 algumas vezes, vá ao MLflow UI, compare os runs 
# e cole o Run ID do seu modelo escolhido abaixo.
BEST_RUN_ID = '5987ef6ab44848009aa19417ffb93ef6'

# --- PASSO 2: O CÓDIGO ABAIXO USA O RUN ID PARA CARREGAR E SALVAR O MODELO ---
# Validação para garantir que o Run ID foi inserido 
if BEST_RUN_ID != '5987ef6ab44848009aa19417ffb93ef6' or not BEST_RUN_ID: 
    raise ValueError("ERRO: Por favor, insira o Run ID do seu melhor modelo na variável BEST_RUN_ID.")


# Constrói o URI do modelo registrado no MLflow 
logged_model_uri = f"runs:/{BEST_RUN_ID}/model" 
print(f"Carregando o melhor modelo do MLflow URI: {logged_model_uri}")

try: 
    # Carrega o pipeline completo (pré-processador + modelo) do MLflow 
    best_pipeline = mlflow.sklearn.load_model(logged_model_uri)
    # Garante que o diretório de destino da aplicação ('app/') exista

    os.makedirs(MODEL_DIR, exist_ok=True)

    # Salva o pipeline para a nossa API no formato .joblib 
    pipeline_path = os.path.join(MODEL_DIR, "pipeline_churn.joblib") 
    joblib.dump(best_pipeline, pipeline_path)

    print(f"\nSUCESSO: Melhor pipeline (do Run ID: {BEST_RUN_ID}) salvo para a API em: '{pipeline_path}'")

except Exception as e: 
    print(f"ERRO ao carregar o modelo do MLflow. Verifique se o Run ID '{BEST_RUN_ID}' está correto.") 
    print(f"Erro detalhado: {e}")




Carregando o melhor modelo do MLflow URI: runs:/5987ef6ab44848009aa19417ffb93ef6/model



SUCESSO: Melhor pipeline (do Run ID: 5987ef6ab44848009aa19417ffb93ef6) salvo para a API em: '..\app\pipeline_churn.joblib'


In [7]:
# ### Célula 7: Treinamento e Salvamento do Explicador SHAP

# Verifica se a variável 'best_pipeline' foi criada com sucesso na célula anterior
if 'best_pipeline' not in locals():
    print("ERRO: A variável 'best_pipeline' não foi encontrada. Por favor, execute a Célula 6 com sucesso antes de rodar esta.")
else:
    # Extrai os componentes do melhor pipeline que foi carregado
    best_model = best_pipeline.named_steps['classifier']
    preprocessor = best_pipeline.named_steps['preprocessor']

    # Aplica o pré-processamento treinado aos dados de treino para o explicador
    X_train_processed = preprocessor.transform(X_train)

    # Recupera os nomes das features após o pré-processamento
    feature_names_processed = preprocessor.get_feature_names_out()

    #print("feature_names_processed",feature_names_processed.tolist())

    #print("X_train_processed:",X_train_processed)

    # Cria o objeto explicador
    print("\nTreinando o explicador SHAP para o melhor modelo selecionado...")
    explainer = shap.TreeExplainer(
        model=best_model,
        data=X_train_processed.toarray(),
        feature_names=feature_names_processed.tolist()
    )
    print("Explicador SHAP treinado com sucesso.")

    # Salva o explicador para ser usado pela nossa API
    explainer_path = os.path.join(MODEL_DIR, "explainer_churn.joblib")
    joblib.dump(explainer, explainer_path)
    print(f"\nSUCESSO: Explicador salvo para a API em: '{explainer_path}'")



Treinando o explicador SHAP para o melhor modelo selecionado...
Explicador SHAP treinado com sucesso.

SUCESSO: Explicador salvo para a API em: '..\app\explainer_churn.joblib'
